## Prepare Data + Model

In [1]:
# !cat examples/data/text_forward.txt

In [2]:
# !ls -al ./outputs/en.1-percent.elmo-bert-causal

In [3]:
import torch
from newlm.lm.elmo.modeling_elmo.elmo_head import ELMOBertLMHeadModel
from newlm.lm.elmo.lm_builder import ELMOLMBuilder
from transformers import BertConfig

#### Model

In [4]:
model = ELMOBertLMHeadModel.from_pretrained(
    "./outputs/en.1-percent.elmo-bert-causal"
) # use pre-trained model

In [5]:
model.eval()
print("Model in eval mode for consistency")

Model in eval mode for consistency


#### Data

In [6]:
%%capture

from newlm.utils.file_util import read_from_yaml
config_file = read_from_yaml('examples/configs/run.1-percent-elmo-bert-causal.yaml')

# lm builder (helper)
elmo_lm_builder = ELMOLMBuilder(
    model_config = config_file['lm']['model']['config'],
    tokenizer="./outputs/en.1-percent.elmo-bert-causal", # use pre-trained tokenizer
    model_type="bert-causal-elmo"
)

# dataset-forward
train_path = "./examples/data/text_forward.txt"
ds_f = elmo_lm_builder._get_dataset(train_path)

2021-11-12 07:48:28.558 | INFO     | newlm.lm.elmo.lm_builder:_get_dataset:142 - Constructing roBERTa style dataset


In [7]:
# trainer (helper)
from transformers import TrainingArguments, Trainer
args = TrainingArguments(output_dir="tmpout",**config_file['lm']['hf_trainer']['args'])

# dataloader-forward
trainer = Trainer(model=model, args=args, data_collator=elmo_lm_builder.data_collator, train_dataset=ds_f,)
dl_f = trainer.get_train_dataloader() # Data Loader-forward

max_steps is given, it will override any value given in num_train_epochs


In [8]:
batch_f = next(iter(dl_f))
batch_f['input_ids'].shape

torch.Size([1, 272])

In [9]:
model.eval()
print("Model in eval mode for consistency")

Model in eval mode for consistency


## Sanity Check

In [10]:
# batch_f

In [11]:
import torch

def reverse_batch(batch_f):
    # reverse input
    batch_f_input = torch.clone(batch_f['input_ids'])
    batch_f_rev_input = torch.cat(
        (
            batch_f_input[0][0:1],
            torch.flip(batch_f_input[0][1:-1], [0]),
            batch_f_input[0][-1:]
        )
    )
    batch_f_rev_input = batch_f_rev_input.reshape(1,-1)

    # reverse labels
    batch_f_rev_labels = torch.clone(batch_f_rev_input)
    
    # batch_rev
    batch_rev = batch_f.copy()
    batch_rev['input_ids'] = batch_f_rev_input
    batch_rev['labels'] = batch_f_rev_labels
    
    return batch_rev

In [20]:
import pandas as pd

def pandas_check(batch_f, batch_rev):
    tokens_f = elmo_lm_builder.tokenizer.convert_ids_to_tokens(batch_f['input_ids'][0])
    tokens_f_rev = elmo_lm_builder.tokenizer.convert_ids_to_tokens(batch_rev['input_ids'][0])
    return pd.DataFrame({"forward": tokens_f, "reverse": tokens_f_rev})

#### Normal vs Reverse

In [13]:
batch_rev = reverse_batch(batch_f)

In [21]:
pandas_check(batch_f, batch_rev)

,forward,reverse
0,[CLS],[CLS]
1,R,.
2,##ø,periods
3,##d,##an
4,##berg,republic
...,...,...
267,republic,##berg
268,##an,##d
269,periods,##ø
270,.,R


In [15]:
batch_f['input_ids'].shape, batch_rev['input_ids'].shape

(torch.Size([1, 272]), torch.Size([1, 272]))

In [16]:
res = model(**batch_f) # forward

l2r_loss tensor(4.8444, grad_fn=<NllLossBackward>)
r2l_loss tensor(8.5702, grad_fn=<NllLossBackward>)


In [17]:
res = model(**batch_rev) # reverse

l2r_loss tensor(9.3590, grad_fn=<NllLossBackward>)
r2l_loss tensor(8.4301, grad_fn=<NllLossBackward>)


#### Random String

In [44]:
# batch_f#['input_ids'].shape

In [63]:
# shuffle data
batch_f_input = batch_f['input_ids']
batch_shuffle_input = torch.cat(
    (
        batch_f_input[0][0:1],
        torch.randint(
            low=5, # 0-4 > ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]']
            high=29999,
            size=(270,),
            dtype=torch.long
        ),
        batch_f_input[0][-1:]
    )
)
batch_shuffle_input = batch_shuffle_input.reshape(1,-1)
# labels
batch_shuffle_labels = torch.clone(batch_shuffle_input)    
# batch_shuffle
batch_shuffle = batch_f.copy()
batch_shuffle['input_ids'] = batch_shuffle_input
batch_shuffle['labels'] = batch_shuffle_labels

In [64]:
batch_shuffle_rev = reverse_batch(batch_shuffle)

##### Trial-1

In [49]:
pandas_check(batch_shuffle, batch_shuffle_rev)

,forward,reverse
0,[CLS],[CLS]
1,stature,swirl
2,sophist,smok
3,dict,Á
4,electrified,##GE
...,...,...
267,##GE,electrified
268,Á,dict
269,smok,sophist
270,swirl,stature


In [50]:
res = model(**batch_shuffle) # forward

l2r_loss tensor(12.9860, grad_fn=<NllLossBackward>)
r2l_loss tensor(11.5442, grad_fn=<NllLossBackward>)


In [51]:
res = model(**batch_shuffle_rev) # reverse

l2r_loss tensor(12.8227, grad_fn=<NllLossBackward>)
r2l_loss tensor(11.2906, grad_fn=<NllLossBackward>)


##### Trial-2

In [60]:
pandas_check(batch_shuffle, batch_shuffle_rev)

,forward,reverse
0,[CLS],[CLS]
1,blew,helmet
2,national,Internet
3,Table,drawn
4,Pomp,tenant
...,...,...
267,tenant,Pomp
268,drawn,Table
269,Internet,national
270,helmet,blew


In [61]:
res = model(**batch_shuffle) # forward

l2r_loss tensor(12.9374, grad_fn=<NllLossBackward>)
r2l_loss tensor(11.5165, grad_fn=<NllLossBackward>)


In [62]:
res = model(**batch_shuffle_rev) # reverse

l2r_loss tensor(12.9639, grad_fn=<NllLossBackward>)
r2l_loss tensor(11.6511, grad_fn=<NllLossBackward>)


##### Trial-3

In [65]:
pandas_check(batch_shuffle, batch_shuffle_rev)

,forward,reverse
0,[CLS],[CLS]
1,hovered,under
2,editing,Ms
3,Whether,guitars
4,##hun,aura
...,...,...
267,aura,##hun
268,guitars,Whether
269,Ms,editing
270,under,hovered


In [66]:
res = model(**batch_shuffle) # forward

l2r_loss tensor(12.9565, grad_fn=<NllLossBackward>)
r2l_loss tensor(11.4332, grad_fn=<NllLossBackward>)


In [67]:
res = model(**batch_shuffle_rev) # reverse

l2r_loss tensor(12.8039, grad_fn=<NllLossBackward>)
r2l_loss tensor(11.5467, grad_fn=<NllLossBackward>)
